In [1]:
!pip show keras

In [2]:
!pip freeze > requirements.txt

In [3]:
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# keras
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
# plt
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
# seed values
SEED = 42
random.seed = SEED
np.random.seed(seed=SEED)

# SETUP
В setup выносим основные настройки

In [5]:
# MODEL
BATCH_SIZE  = 128
EPOCH       = 10
VAL_SPLIT   = 0.15  #15%

# TOKENIZER
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 20000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 150

DATA_PATH = '/kaggle/input/sf-dl-movie-genre-classification/'

# Data
#### TRAIN

In [6]:
train_df = pd.read_csv(DATA_PATH+'train.csv',)

In [7]:
train_df.head()

In [8]:
train_df.info()

In [9]:
train_df.genre.value_counts().plot(kind='bar',figsize=(12,4),fontsize=10)
plt.xticks(rotation=60)
plt.xlabel("Genres",fontsize=10)
plt.ylabel("Counts",fontsize=10)

In [10]:
len(train_df.genre.value_counts())

#### TEST

In [11]:
test_df = pd.read_csv(DATA_PATH+'test.csv')
test_df.head()

In [20]:
test_df.info()

### Target

In [12]:
Y = pd.get_dummies(train_df.genre)
CLASS_NUM = Y.shape[1]
print('Shape of label tensor:', Y.shape)

In [14]:
Y.head()

# Preprocessing

### Clean Data

In [ ]:
# данные у нас и так достаточно чистые

### Tokenize data and convert the text to sequences

Токенизация — это самый первый шаг при обработке текста. Заключается в разбиении (разделении) длинных строк текста в более мелкие: абзацы делим на предложения, предложения на слова. В итоге мы получаем некий словарь, который позволит превратить наш текст в векторное представление для сети.

в Keras есть замечательная утилита [Tokenizer](https://keras.io/preprocessing/text/) которая позволяет сделать всю предобработку буквально в пару строчек кода! 
Больше примеров в коде можно найти тут:  
https://machinelearningmastery.com/prepare-text-data-deep-learning-keras/

In [21]:
# для построения словаря мы используем весь текст, и TRAIN, и TEST
all_text = train_df.text.append(test_df.text, ignore_index=True)

In [22]:
print(type(all_text))
print(len(all_text))

In [23]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(all_text)

теперь переведем наш текст в вектор

In [25]:
%%time
sequences = tokenize.texts_to_sequences(train_df.text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)
print(sequences_matrix.shape)

In [26]:
# вот так теперь выглядит наш текст
print(train_df.text[1])
print(sequences_matrix[1])

# Model
Архитектура сети достаточно простая. Попробуйте самостоятельно поиграться с гиперпараметрами сети и составом слоев. 


In [27]:
def RNN():
    inputs = Input(name='inputs',shape=[MAX_SEQUENCE_LENGTH])
    layer = Embedding(MAX_WORDS,50,input_length=MAX_SEQUENCE_LENGTH)(inputs)
    layer = LSTM(100)(layer)
    layer = Dense(256, activation='relu', name='FC1')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(CLASS_NUM, activation='sigmoid', name='out_layer')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [29]:
model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
history = model.fit(sequences_matrix,Y,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCH,
                    validation_split=VAL_SPLIT)

In [31]:
model.save('keras_nlp_lstm.h5')

In [32]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.show();

In [33]:
plt.title('Accuracy')
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.show();

# Submission

In [34]:
test_sequences = tokenize.texts_to_sequences(test_df.text)
test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [35]:
%%time
predict_proba = model.predict(test_sequences_matrix)

In [36]:
# на соревнованиях всегда сохраняйте predict_proba, чтоб потом можно было построить ансамбль решений
predict_proba = pd.DataFrame(predict_proba, columns=Y.columns)
predict_proba.to_csv('predict_proba.csv', index=False)
predict_proba.head()

In [37]:
predict_genre = Y.columns[np.argmax(predict_proba.values, axis=1)]

In [38]:
submission = pd.DataFrame({'id':range(1, len(predict_genre)+1), 
                           'genre':predict_genre}, 
                          columns=['id', 'genre'])

submission.to_csv('submission.csv', index=False)
submission.head()

Готово!

## Что можно сделать, чтоб улучшить результат:
* Подобрать LR, optimizer, loss
* Поиграться с архитектурой
* Подобрать другие переменные (MAX_WORDS, MAX_SEQUENCE_LENGTH, батч и тп)
* Добавить политику обучения
* Добавить в обучение название фильма (для pro: попробуйте это сделать через concatenate слой)
* Попробовать уже предобученные Векторизаторы (GLOVE, W2V и тд)
* Попробовать уже предобученные модели (BERT, TRANFORMER и тд)
* Построить ансамбль из разных архитектур

### Удачи в соревновании!